# Becoming a Python Guru


<img src="https://imgs.xkcd.com/comics/random_number.png" style="width: 800px;"/>


## Lecture today:

* Essential tools to write better Python
 * List comprehensions
 * Iterators
 * Decorators
 * Good documentation 
* Error handling

  * Python3 Type hints
* Testing your code
* Structuring your project
    * Modules and setup.py files    


 **Why document your code?**

Of paramount importance to produce good code **understandable by others and the author**, and hence maintainable in the long run.

If you want to learn more on how to write comments, I recommend reading this blog post: http://antirez.com/news/124